# Notebook for checking that my data is what I expect

In [1]:
from __future__ import division, print_function, absolute_import

import time
import random
import glob
import os.path
import textwrap

from IPython.display import display, HTML, Image, Latex

%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
plt.rcParams['figure.figsize'] = 8, 8
import seaborn as sns
sns.set_style('white')

import numpy as np
import pandas as pd
import scipy.optimize
import scipy.spatial
import scipy.stats
import cv2
from tf import transformations

In [2]:
data = np.load("../data/board_data.npz")

In [28]:
df = pd.DataFrame(columns=[
    "bag_files", 
    "image_coords", 
    "object_coords", 
    "cam_rb_poses", 
    "cam_marker_counts", 
    "cal_markers_counts", 
    "cal_markers",
])
df.bag_files = np.array([x.strip().split("/")[-1] for x in data["bag_files"]])
df.image_coords = data["image_coordinates"]
print(data["image_coordinates"].shape)
print(data["object_coordinates"].shape)
print(data["camera_rb_poses"].shape)
# df.object_coords = data["object_coordinates"]
# df.cam_rb_poses = data["camera_rb_poses"]
df.cal_markers_counts = data["calibration_object_marker_counts"]
df.cam_marker_counts = data["camera_marker_counts"]




(70,)
(70, 187, 3)
(70, 4, 4)


In [27]:
df

,bag_files,image_coords,object_coords,cam_rb_poses,cam_marker_counts,cal_markers_counts,cal_markers
0,13_2018-08-25-17-08-32.bag,"[[987.5, 202.5], [1021.42, 215.489], [1005.76,...",NaN,NaN,5,5,NaN
1,13_2018-08-25-17-08-32.bag,"[[987.507, 202.458], [1021.39, 215.535], [1005...",NaN,NaN,5,5,NaN
2,13_2018-08-25-17-08-32.bag,"[[1036.86, 186.618], [1003.46, 173.907], [1070...",NaN,NaN,5,5,NaN
3,13_2018-08-25-17-08-32.bag,"[[987.644, 202.648], [1021.42, 215.489], [1005...",NaN,NaN,5,5,NaN
4,13_2018-08-25-17-08-32.bag,"[[987.507, 202.458], [1021.42, 215.489], [1005...",NaN,NaN,5,5,NaN
5,13_2018-08-25-17-08-32.bag,"[[987.531, 202.563], [1021.42, 215.489], [1005...",NaN,NaN,5,5,NaN
6,13_2018-08-25-17-08-32.bag,"[[987.634, 202.582], [1021.43, 215.441], [1005...",NaN,NaN,5,5,NaN
7,13_2018-08-25-17-08-32.bag,"[[987.639, 202.553], [1021.38, 215.53], [1005....",NaN,NaN,5,5,NaN
8,13_2018-08-25-17-08-32.bag,"[[987.507, 202.458], [1021.43, 215.337], [1005...",NaN,NaN,5,5,NaN
9,13_2018-08-25-17-08-32.bag,"[[987.649, 202.62], [1021.42, 215.489], [1005....",NaN,NaN,5,5,NaN
